# **Lunar Lander con Q-Learning**

### **1. Bibliotecas**

In [1]:
#!pip3 install cmake gymnasium scipy numpy gymnasium[box2d] pygame==2.6.0 swig
# Tal vez tengan que ejecutar lo siguiente en sus máquinas (ubuntu 20.04)
# sudo apt-get remove swig
# sudo apt-get install swig3.0
# sudo ln -s /usr/bin/swig3.0 /usr/bin/swig
# En windows tambien puede ser necesario MSVC++

import sys
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import random
import pygame
from pygame.locals import *

### **2. Jugando a mano**

A continuación se puede jugar un episodio del lunar lander. Se controlan los motores con el teclado. Notar que solo se puede realizar una acción a la vez (que es parte del problema), y que en esta implementación, izq toma precedencia sobre derecha, que toma precedencia sobre el motor principal.

In [6]:
# Inicializar pygame (para el control con el teclado) y el ambiente
pygame.init()
env = gym.make('LunarLander-v2', render_mode='human')
env.reset()
pygame.display.set_caption('Lunar Lander')

clock = pygame.time.Clock()
done = False

while not done:
    for event in pygame.event.get():
        if event.type == QUIT:
            done = True
            break

    keys = pygame.key.get_pressed()

    # Map keys to actions
    if keys[K_LEFT]:
        action = 3  # Fire left orientation engine
    elif keys[K_RIGHT]:
        action = 1 # Fire right orientation engine
    elif keys[K_UP]:
        action = 2  # Fire main engine
    else:
        action = 0  # Do nothing

    _, _, terminated, truncated, _ = env.step(action)
    env.render()
    clock.tick(10)

    if terminated or truncated:
        done = True

env.close()
pygame.quit()

## **3. Discretizando el estado**

El estado consiste de posiciones y velocidades en (x,y,theta) y en información de contacto de los pies con la superficie.

Como varios de estos son continuos, tenemos que discretizarlos para aplicar nuestro algoritmo de aprendizaje por refuerzo tabular.

In [2]:
# Cuántos bins queremos por dimensión
# Pueden considerar variar este parámetro
bins_per_dim = 20

#          Estado:
#          (x,            y,            x_vel,        y_vel,        theta,        theta_vel,    pie_izq_en_contacto, pie_derecho_en_contacto)
NUM_BINS = [bins_per_dim, bins_per_dim, bins_per_dim, bins_per_dim, bins_per_dim, bins_per_dim, 2,                   2]

env = gym.make('LunarLander-v2', render_mode='human')
env.reset()

# Tomamos los rangos del env
OBS_SPACE_HIGH = env.observation_space.high
OBS_SPACE_LOW = env.observation_space.low
OBS_SPACE_LOW[1] = 0 # Para la coordenada y (altura), no podemos ir más abajo que la zona de aterrizaje (que está en el 0, 0)

# Los bins para cada dimensión
bins = [
    np.linspace(OBS_SPACE_LOW[i], OBS_SPACE_HIGH[i], NUM_BINS[i] - 1)
    for i in range(len(NUM_BINS) - 2) # last two are binary
]
# Se recomienda observar los bins para entender su estructura
# print ("Bins: ", bins)

def discretize_state(state, bins):
    """Discretize the continuous state into a tuple of discrete indices."""
    state_disc = list()
    for i in range(len(state)):
        if i >= len(bins):  # For binary features (leg contacts)
            state_disc.append(int(state[i]))
        else:
            state_disc.append(
                np.digitize(state[i], bins[i])
            )
    return tuple(state_disc)

In [3]:
# Ejemplos
print(discretize_state([0.0, 0.0, 0, 0, 0, 0, 1, 1], bins)) # En la zona de aterrizaje y quieto
print(discretize_state([0, 1.5, 0, 0, 0, 0, 0, 0], bins)) # Comenzando la partida, arriba y en el centro

(10, 1, 10, 10, 10, 10, 1, 1)
(10, 19, 10, 10, 10, 10, 0, 0)


## **4. Agentes y la interacción con el entorno**

Vamos a definir una interfaz para nuestro agente:

In [4]:
class Agente:
    def elegir_accion(self, estado, max_accion, explorar = True) -> int:
        """Elegir la accion a tomar en el estado actual y el espacio de acciones
            - estado_anterior: el estado desde que se empezó
            - estado_siguiente: el estado al que se llegó
            - accion: la acción que llevo al agente desde estado_anterior a estado_siguiente
            - recompensa: la recompensa recibida en la transicion
            - terminado: si el episodio terminó
        """
        pass

    def aprender(self, estado_anterior, estado_siguiente, accion, recompensa, terminado):
        """Aprender a partir de la tupla 
            - estado_anterior: el estado desde que se empezó
            - estado_siguiente: el estado al que se llegó
            - accion: la acción que llevo al agente desde estado_anterior a estado_siguiente
            - recompensa: la recompensa recibida en la transicion
            - terminado: si el episodio terminó en esta transición
        """
        pass

Para un agente aleatorio, la implementación sería:

In [5]:
import random

class AgenteAleatorio(Agente):
    def elegir_accion(self, estado, max_accion, explorar = True) -> int:
        # Elige una acción al azar
        return random.randrange(max_accion)

    def aprender(self, estado_anterior, estado_siguiente, accion, recompensa, terminado):
        # No aprende
        pass

Luego podemos definir una función para ejecutar un episodio con un agente dado:

In [32]:
def ejecutar_episodio(agente, aprender = True, render = None):
    entorno = gym.make('LunarLander-v2', render_mode=render).env

    iteraciones = 0
    recompensa_total = 0

    termino = False
    truncado = False
    estado_anterior, info = entorno.reset()
    
    while not termino and not truncado:
        # Le pedimos al agente que elija entre las posibles acciones (0..entorno.action_space.n)
        # Si no estamos aprendiendo, explotamos sin explorar
        accion = agente.elegir_accion(estado_anterior, entorno.action_space.n, aprender)
        # Realizamos la accion
        estado_siguiente, recompensa, termino, truncado, info = entorno.step(accion)
        # Le informamos al agente para que aprenda
        if (aprender):
            agente.aprender(estado_anterior, estado_siguiente, accion, recompensa, termino)

        estado_anterior = estado_siguiente
        iteraciones += 1
        recompensa_total += recompensa
    env.close()
    return recompensa_total

In [17]:
# Ejecutamos un episodio con el agente aleatorio y modo render 'human', para poder verlo
ejecutar_episodio(AgenteAleatorio(), render = 'human')

-94.44971519408841

Podemos ejecutar este ambiente muchas veces y tomar métricas al respecto

In [ ]:
agente = AgenteAleatorio()
recompensa_episodios = []

exitos = 0
num_episodios = 100
for i in range(num_episodios):
    recompensa = ejecutar_episodio(agente)
    # Los episodios se consideran exitosos si se obutvo 200 o más de recompensa total
    if (recompensa >= 200):
        exitos += 1
    recompensa_episodios += [recompensa]

import numpy
print(f"Tasa de éxito: {exitos / num_episodios}. Se obtuvo {numpy.mean(recompensa_episodios)} de recompensa, en promedio")

### **5. Programando un agente que aprende**

La tarea a realizar consiste en programar un agente de aprendizaje por refuerzos:

In [78]:
import numpy as np
import random

# Hiperparametros
#   Politica de exploracion
#   Temperatura / k
#   Alfa / Learning rate
#   Aprender en el momento / Aprender al final
#   Cantidad de bins

class AgenteRL(Agente):

    def __init__(self) -> None:
        super().__init__()
        self.Q = {}

    def elegir_accion(self, estado, max_accion, explorar=True) -> int:
        estado = tuple(estado)

        if explorar:
            # Explorar
            if random.random() < 0.5 or estado not in self.Q:
                return random.randrange(max_accion)
            else:
                accion = None
                for a in self.Q[estado]:  
                    if accion is None or self.Q[estado][a] > self.Q[estado][accion]:
                        accion = a
                return accion  
        else:
            # Explotar
            accion = None
            if estado not in self.Q:
                return random.randrange(max_accion)
            else:
                for a in self.Q[estado]:
                    if accion is None or self.Q[estado][a] > self.Q[estado][accion]:
                        accion = a
                return accion  

    def aprender(self, estado_anterior, estado_actual, accion, recompensa, terminado):
        estado_anterior = tuple(estado_anterior)
        estado_actual = tuple(estado_actual)

        if estado_anterior not in self.Q:
            self.Q[estado_anterior] = {}
            for i in range(4):
                self.Q[estado_anterior][i] = 0

        if estado_actual not in self.Q:
            self.Q[estado_actual] = {}
            for i in range(4):
                self.Q[estado_actual][i] = 0
        
        self.Q[estado_anterior][accion] = recompensa + max(self.Q[estado_actual].values())


    def fin_episodio(self):
        pass

Y ejecutar con el muchos episodios:

In [79]:
# Nota: hay que transformar esta celda en código para ejecutar (Esc + y)
# Advertencia: este bloque es un loop infinito si el agente se deja sin implementar

entorno = gym.make('LunarLander-v2').env
agente = AgenteRL()
exitos = 0
recompensa_episodios = []
num_episodios = 100
for i in range(num_episodios):
    recompensa = ejecutar_episodio(agente, aprender = True, render='human')
    # Los episodios se consideran exitosos si se obutvo 200 o más de recompensa total
    if (recompensa >= 200):
        exitos += 1
    recompensa_episodios += [recompensa]
    #imprimir promedio
    print(np.mean(recompensa_episodios))
print(f"Tasa de éxito: {exitos / num_episodios}. Se obtuvo {numpy.mean(recompensa_episodios)} de recompensa, en promedio")

-100.28373216461641
-147.40921174053074
-162.48869697419002
-154.01950268553014
-145.6701159404691
-144.8472734417505
-142.7383339959325
-142.7755334479788
-140.0421668031646
-150.36475879790976
-148.158263975976
-148.31894503940177
-146.49789989144023
-134.67991751030056
-138.43736332843488
-137.09265521637673
-142.79223615675278
-144.29367656695666
-140.5095654633999
-146.58338204644411
-153.93852839799547
-157.25454779413673
-155.82763426247098
-155.22646880256877
-165.8617531606594
-162.73891867918658
-166.23937645198086
-168.61632618788775
-166.65054284781883


KeyboardInterrupt: 

Analizar los resultados de la ejecución anterior, incluyendo:
 * Un análisis de los parámetros utilizados en el algoritmo (aprendizaje, política de exploración)
 * Un análisis de algunos 'cortes' de la matriz Q y la política (p.e. qué hace la nave cuando está cayendo rápidamente hacia abajo, sin rotación)
 * Un análisis de la evolución de la recompensa promedio
 * Un análisis de los casos de éxito
 * Un análisis de los casos en el que el agente falla
 * Qué limitante del agente de RL les parece que afecta más negativamente su desempeño. Cómo lo mejorarían? 

In [22]:
# Analizar los resultados aqui